## Analyze trial results

In [3]:
import pandas as pd
import os, sys
import os.path
import numpy as np
import scipy
import scipy.stats

In [44]:
run_output_dir = '2018-05-28--02.27.37-686644481__D2a__moredata'
file_signature = 'populations.csv'

In [48]:
trial_end_date = 1000 #the last normal recruit completes on day 958 

In [50]:
pops_file = [s for s in os.listdir(run_output_dir) if s.find(file_signature)>-1][0]

### Study details

In [77]:
header_output = pd.read_csv(os.path.join(run_output_dir, pops_file), nrows=1, header=1)
print(header_output[['vaccine_schedule', 'initial_pwid_count', 'vaccine_study_arm_n', 'vaccine_annual_loss', 'vaccine_dose2_day', 'vaccine_dose3_day', 'vaccine_enrollment_duration_days', 'vaccine_enrollment_launch_day', 'vaccine_enrollment_probability_positiveinnetwork', 'vaccine_enrollment_probability_unbiased', 'vaccine_followup1_periods', 'vaccine_followup2_periods', 'vaccine_followup_weeks']
])

  vaccine_schedule  initial_pwid_count  vaccine_study_arm_n  \
0              D2a               32000                  797   

   vaccine_annual_loss  vaccine_dose2_day  vaccine_dose3_day  \
0                  0.2               60.0              180.0   

   vaccine_enrollment_duration_days  vaccine_enrollment_launch_day  \
0                               365                           30.0   

   vaccine_enrollment_probability_positiveinnetwork  \
0                                               1.0   

   vaccine_enrollment_probability_unbiased  vaccine_followup1_periods  \
0                                      0.0                         18   

   vaccine_followup2_periods  vaccine_followup_weeks  
0                          9                     4.0  


### Populations

In [52]:
pops_output = pd.read_csv(os.path.join(run_output_dir, pops_file), header=3)

In [53]:
pops_output.columns

Index(['BurnInMode', 'Tick', 'Year', 'incidence_daily', 'aggregate_courses',
       'recr_study_aggregate_vaccine', 'cmpl_study_chronic_aggregate_vaccine',
       'recr_placebo_aggregate_vaccine', 'losses_daily',
       'cmpl_study_notchronic_aggregate_vaccine',
       ...
       'fraction_VaccineArm=noarm', 'hcvabpos_VaccineArm=noarm',
       'infected_VaccineArm=noarm', 'chronic_VaccineArm=noarm',
       'population_VaccineArm=noarm', 'ABpreval_VaccineArm=noarm',
       'RNApreval_VaccineArm=noarm', 'intreatment_VaccineArm=noarm',
       'vaccinetrial_VaccineArm=noarm', 'Unnamed: 304'],
      dtype='object', length=305)

### Total enrolled

In [56]:
enrolled_pops_fields = ['recr_study_aggregate_vaccine', 'recr_placebo_aggregate_vaccine']
enrolled_pops=np.array(pops_output.loc[trial_end_date, enrolled_pops_fields])
pops_output.loc[trial_end_date, ['Tick'] + enrolled_pops_fields]

Tick                              1001.001
recr_study_aggregate_vaccine       773.000
recr_placebo_aggregate_vaccine     768.000
Name: 1000, dtype: float64

### Still in follow-up (should be small)

In [65]:
active_pops_fields = ['vaccinetrial_VaccineArm=study', 'vaccinetrial_VaccineArm=placebo']
active_pops=np.array(pops_output.loc[trial_end_date, active_pops_fields])
pops_output.loc[trial_end_date, ['Tick'] + active_pops_fields]

Tick                               1001.001
vaccinetrial_VaccineArm=study         4.000
vaccinetrial_VaccineArm=placebo       6.000
Name: 1000, dtype: float64

### Infection status

In [58]:
## note: cmpl_ variables are accumulators inside the simulation, and are valid even if individuals leave the simulation

#### chronic

In [59]:
chronic_pops=np.array(pops_output.loc[trial_end_date, ['cmpl_study_chronic_aggregate_vaccine', 'cmpl_placebo_chronic_aggregate_vaccine']])
pops_output.loc[trial_end_date, ['Tick', 'cmpl_study_chronic_aggregate_vaccine', 'cmpl_placebo_chronic_aggregate_vaccine']]

Tick                                      1001.001
cmpl_study_chronic_aggregate_vaccine        20.000
cmpl_placebo_chronic_aggregate_vaccine      28.000
Name: 1000, dtype: float64

#### not chronic

In [66]:
nonchronic_pops=np.array(pops_output.loc[trial_end_date, ['cmpl_study_notchronic_aggregate_vaccine', 'cmpl_placebo_notchronic_aggregate_vaccine']])
pops_output.loc[trial_end_date, ['Tick', 'cmpl_study_notchronic_aggregate_vaccine', 'cmpl_placebo_notchronic_aggregate_vaccine']]

Tick                                         1001.001
cmpl_study_notchronic_aggregate_vaccine       553.000
cmpl_placebo_notchronic_aggregate_vaccine     512.000
Name: 1000, dtype: float64

### Total completed trial per protocol

In [69]:
total_completed = chronic_pops + nonchronic_pops
print('Study: %.0f, Placebo: %.0f'%(total_completed[0], total_completed[1]))

Study: 573, Placebo: 540


Note: it's a minor mystery why they are slightly unbalanced.  The

### Results

In [80]:
trial_contingency_table = np.reshape(np.concatenate((nonchronic_pops, chronic_pops), axis=0), newshape=(2,2))

In [81]:
chi2, p, dof, expected = scipy.stats.chi2_contingency(trial_contingency_table)
chi2, p, expected

(1.5461032705955524, 0.21371162686538592, array([[ 548.2884097,  516.7115903],
        [  24.7115903,   23.2884097]]))

## Vaccine efficacy:
$$ VE_𝑎 = 1 − 𝑎_{vaccine}/𝑎_{placebo} $$

In our case, we are interested in the prevention of chronic infection.  Therefore, we calculate:
$a_{vaccine} = $(HCV chronic vaccinees) / (total vaccinees)

and similarly for the placebo group

In [75]:
a_vaccine = (chronic_pops[0] / total_completed[0])
a_vaccine

0.034904013961605584

In [72]:
a_placebo = (chronic_pops[1] / total_completed[1])
a_placebo

0.05185185185185185

In [76]:
1 - (a_vaccine/a_placebo)

0.32685115931189224

# Conclusions

* The trial failed to achieve statistically sig reduction
* The trial failed to achieve 80% reduction in chronicity, attaining only 30%